### Notebook 3 - Translate RCT Output to align with Inventory Data

This notebook takes the parsed RCT output from the previous notebooks and aligns it with the inventory data. It also transfers the Tree ID from the inventory to the RCT data using a spatial join.

In [ ]:
# Suppress depreciation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Import the required modules
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import helper_functions

### Load in Inventory data

In [ ]:
# Read in as pandas dataframe
inventory_df = pd.read_csv('../data/inventory_data.csv')

# Filter to only include the data for Mak1
inventory_df = inventory_df[inventory_df['plot_id'] == 'Mak1']

inventory_df

### Convert the inventory tree locations to Cartesian coordinates

In [ ]:
inventory_df['x'] = inventory_df['distance'] * np.sin(np.radians(inventory_df['bearing']))
inventory_df['y'] = inventory_df['distance'] * np.cos(np.radians(inventory_df['bearing']))

#### Load in the RCT data

In [ ]:
rct_df = pd.read_csv('../data/rct_tree_data.csv')
rct_df

In [ ]:
# Plot the RCT tree locations along with inventory data
plt.figure(figsize=(12, 12))

# Scatter plot for tree_merged_df
plt.scatter(rct_df['x'], rct_df['y'], alpha=0.5, label='RCT Trees')

# Scatter plot for inventory_df
plt.scatter(inventory_df['x'], inventory_df['y'], alpha=0.5, label='Inventory Trees')

# Adding tree ID labels for inventory_df points
for i in range(inventory_df.shape[0]):
    plt.text(inventory_df['x'][i], inventory_df['y'][i], f"{inventory_df['tree_number'][i]}")

# Adding labels and grid
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.grid(True)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.legend()
plt.show()

### Perfom translation of RCT data to inventory using Open3D iterative closest point algorthim.

In [ ]:
# Convert dataframes to numpy arrays of points
source_points = rct_df[['x', 'y']].to_numpy()
target_points = inventory_df[['x', 'y']].to_numpy()

# Ensure points are in the correct shape (Nx3) by adding a z dimension of 0
source_points = np.hstack((source_points, np.zeros((source_points.shape[0], 1))))
target_points = np.hstack((target_points, np.zeros((target_points.shape[0], 1))))

# Initial rotation matrix
angle = np.radians(10)  # Example: 10 degrees
rotation = helper_functions.R.from_euler('z', angle, degrees=False).as_matrix()
init_pose = np.eye(4)
init_pose[:3, :3] = rotation

# Run ICP
T, _ = helper_functions.icp(source_points, target_points, init_pose)

# Apply the final transformation to the source points
source_points_transformed = (source_points @ T[:3, :3].T) + T[:3, 3]

# Update the DataFrame with the transformed coordinates
rct_df['x_translated'] = source_points_transformed[:, 0]
rct_df['y_translated'] = source_points_transformed[:, 1]

print("Transformation Matrix:\n", T)
print(rct_df)


There are many ways we could go about aligning the RCT tree locations with the inventory. One option would be to use CloudCompare, in some cases it may be  better to visualise and iteratively adjust the translation.

### Plot the updated RCT stem locations, along with the inventory data. 

In [ ]:
# Calculate stem area
rct_df['stem_area'] = np.pi * (rct_df['DBH'] ** 2) * 100

# Plot the RCT tree locations along with inventory data
plt.figure(figsize=(12, 12))

# Scatter plot for tree_merged_df
plt.scatter(rct_df['x_translated'], rct_df['y_translated'], alpha=0.5, label='RCT Trees')

# Scatter plot for inventory_df
plt.scatter(inventory_df['x'], inventory_df['y'], alpha=0.5, label='Inventory Trees')

# Adding tree ID labels for inventory_df points
for i in range(inventory_df.shape[0]):
    plt.text(inventory_df['x'][i], inventory_df['y'][i], f"{inventory_df['tree_number'][i]}")

# Adding labels and grid
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.grid(True)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.legend()
plt.show()

### Perform a spatial join to transfer the tree ID from the inventory data to the RCT data.

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Create GeoDataFrames from pandas
translated_tree_merged_gdf = gpd.GeoDataFrame(
    rct_df,
    geometry=[Point(xy) for xy in zip(rct_df['x_translated'], rct_df['y_translated'])])

inventory_gdf = gpd.GeoDataFrame(
    inventory_df,
    geometry=[Point(xy) for xy in zip(inventory_df['x'], inventory_df['y'])])

# Perform spatial join
joined_gdf = gpd.sjoin_nearest(
    translated_tree_merged_gdf, 
    inventory_gdf[['geometry', 'tree_number']],  # Include only geometry and tree_number for joining
    how="inner", 
    distance_col="distance"
)

# Rename tree_id to indicate its source, if needed
joined_gdf.rename(columns={'tree_number': 'closest_inv_id'}, inplace=True)

# Convert the geo dataframe to a pandas dataframe, removing the geometry column and right index
joined_df = pd.DataFrame(joined_gdf.drop(columns=['geometry','index_right']))

# Save the joined dataframe to a CSV file
joined_df.to_csv('../data/rct_translated_joined_data.csv', index=False)

joined_df

### Finally, we can plot the RCT data with the joined inventory tree IDs

In [ ]:
# Plot the RCT tree locations along with inventory data
plt.figure(figsize=(12, 12))

# Scatter plot for tree_merged_df
plt.scatter(joined_df['x_translated'], joined_df['y_translated'], alpha=0.5, label='RCT Trees')

# Scatter plot for inventory_df
plt.scatter(inventory_df['x'], inventory_df['y'], alpha=0.5, label='Inventory Trees')

# Adding tree ID labels for inventory_df points
for i in range(joined_df.shape[0]):
    plt.text(joined_df['x_translated'][i], joined_df['y_translated'][i], f"{int(joined_df['closest_inv_id'][i])}, dist:{joined_df['distance'][i]:.2f}")

# Adding labels and grid
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.grid(True)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.legend()
plt.show()